In [3]:
import pandas as pd
import os
import sklearn as sk
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from scipy.sparse import csr_matrix
import implicit
from sklearn.metrics.pairwise import pairwise_distances


In [4]:
os.getcwd()

'C:\\Users\\enabi\\OneDrive\\Desktop\\11182022\\DS_apply\\Projects_for_github\\Recommonder_system\\Recommender_system'

In [5]:
df_user_ratedmovies_timestamps= pd.read_csv('all/user_ratedmovies-timestamps.dat', sep='\t', encoding='ISO-8859-1')

In [6]:
df_user_ratedmovies_timestamps.head()

,userID,movieID,rating,timestamp
0,75,3,1.0,1162160236000
1,75,32,4.5,1162160624000
2,75,110,4.0,1162161008000
3,75,160,2.0,1162160212000
4,75,163,4.0,1162160970000


In [7]:
df_user_ratedmovies_timestamps['movieID'].max()

65133

In [8]:
df_movies= pd.read_csv('all/movies.dat', sep='\t', encoding='ISO-8859-1')

In [9]:
df_movies.head()

,id,title,imdbID,spanishTitle,imdbPictureURL,year,rtID,rtAllCriticsRating,rtAllCriticsNumReviews,rtAllCriticsNumFresh,...,rtAllCriticsScore,rtTopCriticsRating,rtTopCriticsNumReviews,rtTopCriticsNumFresh,rtTopCriticsNumRotten,rtTopCriticsScore,rtAudienceRating,rtAudienceNumRatings,rtAudienceScore,rtPictureURL
0,1,Toy story,114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995,toy_story,9,73,73,...,100,8.5,17,17,0,100,3.7,102338,81,http://content7.flixster.com/movie/10/93/63/10...
1,2,Jumanji,113497,Jumanji,http://ia.media-imdb.com/images/M/MV5BMzM5NjE1...,1995,1068044-jumanji,5.6,28,13,...,46,5.8,5,2,3,40,3.2,44587,61,http://content8.flixster.com/movie/56/79/73/56...
2,3,Grumpy Old Men,107050,Dos viejos gruñones,http://ia.media-imdb.com/images/M/MV5BMTI5MTgy...,1993,grumpy_old_men,5.9,36,24,...,66,7,6,5,1,83,3.2,10489,66,http://content6.flixster.com/movie/25/60/25602...
3,4,Waiting to Exhale,114885,Esperando un respiro,http://ia.media-imdb.com/images/M/MV5BMTczMTMy...,1995,waiting_to_exhale,5.6,25,14,...,56,5.5,11,5,6,45,3.3,5666,79,http://content9.flixster.com/movie/10/94/17/10...
4,5,Father of the Bride Part II,113041,Vuelve el padre de la novia (Ahora también abu...,http://ia.media-imdb.com/images/M/MV5BMTg1NDc2...,1995,father_of_the_bride_part_ii,5.3,19,9,...,47,5.4,5,1,4,20,3,13761,64,http://content8.flixster.com/movie/25/54/25542...


In [10]:
df_movies['id'].max()

65133

In [14]:
#Obtain the ratings corresponding to some movies:

some_movies= ['The Matrix', 'Alien', 'The Godfather']
some_ids= {}
for mov in some_movies:
    some_ids[mov]=df_movies.loc[df_movies['title']==mov,'id'].values
some_ids

{'The Matrix': array([2571], dtype=int64),
 'Alien': array([1214], dtype=int64),
 'The Godfather': array([  858, 25934], dtype=int64)}

In [123]:
df_movies.loc[df_movies['id'] .isin ([2571, 1214, 858, 25934])]

,id,title,imdbID,spanishTitle,imdbPictureURL,year,rtID,rtAllCriticsRating,rtAllCriticsNumReviews,rtAllCriticsNumFresh,...,rtAllCriticsScore,rtTopCriticsRating,rtTopCriticsNumReviews,rtTopCriticsNumFresh,rtTopCriticsNumRotten,rtTopCriticsScore,rtAudienceRating,rtAudienceNumRatings,rtAudienceScore,rtPictureURL
787,858,The Godfather,68646,El padrino,http://ia.media-imdb.com/images/M/MV5BMTc3MDQz...,1972,godfather,9,66,66,...,100,8.4,11,11,0,100,4.4,172072,97,http://content9.flixster.com/movie/11/12/76/11...
1102,1214,Alien,78748,Alien - El octavo pasajero,http://ia.media-imdb.com/images/M/MV5BMTgzMDk3...,1979,american-alien,0,0,0,...,0,0,0,0,0,0,0,0,0,http://content7.flixster.com/movie/10/87/47/10...
2350,2571,The Matrix,133093,Matrix,http://ia.media-imdb.com/images/M/MV5BMjEzNjg1...,1999,matrix,7.4,125,108,...,86,6.2,28,19,9,67,3.6,207333,81,http://content7.flixster.com/movie/16/90/52/16...
8048,25934,The Godfather,68646,El padrino,http://ia.media-imdb.com/images/M/MV5BMTc3MDQz...,1972,godfather,9,66,66,...,100,8.4,11,11,0,100,4.4,172072,97,http://content9.flixster.com/movie/11/12/76/11...


In [124]:
#Obtain the ratings of 'Alien':
r_Alien=df_user_ratedmovies_timestamps.loc[df_user_ratedmovies_timestamps['movieID']==1214, 'rating'].mean()
r_Alien

3.9579395085066165

In [125]:
#Obtain the ratings of 'The Godfather':
r_Godfather=df_user_ratedmovies_timestamps.loc[df_user_ratedmovies_timestamps['movieID'].isin([25934,858]), 'rating'].mean()
r_Godfather

4.334507042253521

In [126]:
#Obtain the ratings of 'The Matrix':
r_Matrix=df_user_ratedmovies_timestamps.loc[df_user_ratedmovies_timestamps['movieID']==2571, 'rating'].mean()
r_Matrix

4.173952095808383

In [127]:
# Obtain the users rated 'The Godfather'

user_Godfather =df_user_ratedmovies_timestamps.loc[df_user_ratedmovies_timestamps['movieID'].isin([25934,858]), 'userID'].values

user_Godfather

array([   78,   175,   190, ..., 71509, 71525, 71534], dtype=int64)

In [128]:
# Obtain the users who rated 'Alien'

user_Alien =df_user_ratedmovies_timestamps.loc[df_user_ratedmovies_timestamps['movieID']==1214, 'userID'].values

user_Alien

array([   78,   175,   190, ..., 71483, 71497, 71525], dtype=int64)

In [129]:
# Obtain the users who rated 'The Matrix'

user_Matrix =df_user_ratedmovies_timestamps.loc[df_user_ratedmovies_timestamps['movieID']==2571, 'userID'].values

user_Matrix

array([   75,    78,   170, ..., 71525, 71529, 71534], dtype=int64)

In [130]:
# Obtain the users who rated the previous movies

distinct_users =df_user_ratedmovies_timestamps.loc[df_user_ratedmovies_timestamps['movieID'].isin([2571, 1214, 858, 25934]), 'userID'].unique()

distinct_users


array([   75,    78,   170, ..., 71525, 71529, 71534], dtype=int64)

In [131]:
# Create a sparse matrix -- This matrix will include each item as a row, and the users will be the columns
# The value of each cell of the matrix will be filled with the user rating for each movie

df_mov= df_user_ratedmovies_timestamps.loc[df_user_ratedmovies_timestamps['movieID'].isin([2571, 1214, 858, 25934]), :]
df_mov

,userID,movieID,rating,timestamp
29,75,2571,4.5,1162161050000
98,78,858,4.0,1089332870000
149,78,1214,5.0,1083963994000
286,78,2571,5.0,1083964044000
612,170,2571,3.5,1162208300000
...,...,...,...,...
854959,71525,1214,5.0,1045440408000
855027,71525,2571,5.0,1001839814000
855396,71529,2571,2.5,1162098860000
855461,71534,858,5.0,1196645187000


In [132]:
matrix = df_mov.pivot(index='movieID', columns='userID', values='rating')

matrix

userID,75,78,170,175,190,267,325,383,476,477,...,71331,71420,71478,71483,71487,71497,71509,71525,71529,71534
movieID,,,,,,,,,,,,,,,,,,,,,
858,NaN,4.0,NaN,5.0,1.0,2.5,5.0,4.0,NaN,5.0,...,NaN,4.0,5.0,5.0,5.0,5.0,5.0,5.0,NaN,5.0
1214,NaN,5.0,NaN,5.0,3.5,4.0,5.0,4.0,4.5,4.5,...,NaN,3.0,NaN,4.0,NaN,4.0,NaN,5.0,NaN,NaN
2571,4.5,5.0,3.5,5.0,5.0,4.0,NaN,4.0,4.5,4.5,...,5.0,5.0,NaN,4.0,5.0,5.0,3.5,5.0,2.5,4.5
25934,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [133]:
# Since both 25934 and 858 are the movieId of the same movie: "The godfather"
# I want to merge them, first see what is in the movieId==25934, because it seems more sparse than the other one.

matrix.loc[25934].value_counts(dropna=False)

NaN    1820
3.5       1
4.5       1
Name: 25934, dtype: int64

In [134]:
# Now, we can transfer these two values to the movieId==858 (If they are not already full) and delete the movieId==25934 row.

filt=matrix.loc[25934].notna()
matrix.columns[filt]

Int64Index([51033, 52983], dtype='int64', name='userID')

In [135]:
matrix.loc[858, [51033, 52983]]

userID
51033    4.0
52983    5.0
Name: 858, dtype: float64

In [136]:
# I deleted the row movieId==25934

matrix= matrix.drop(25934, axis=0)

In [137]:
matrix

userID,75,78,170,175,190,267,325,383,476,477,...,71331,71420,71478,71483,71487,71497,71509,71525,71529,71534
movieID,,,,,,,,,,,,,,,,,,,,,
858,NaN,4.0,NaN,5.0,1.0,2.5,5.0,4.0,NaN,5.0,...,NaN,4.0,5.0,5.0,5.0,5.0,5.0,5.0,NaN,5.0
1214,NaN,5.0,NaN,5.0,3.5,4.0,5.0,4.0,4.5,4.5,...,NaN,3.0,NaN,4.0,NaN,4.0,NaN,5.0,NaN,NaN
2571,4.5,5.0,3.5,5.0,5.0,4.0,NaN,4.0,4.5,4.5,...,5.0,5.0,NaN,4.0,5.0,5.0,3.5,5.0,2.5,4.5


In [138]:
matrix_zero = matrix.copy()

In [139]:
matrix_zero

userID,75,78,170,175,190,267,325,383,476,477,...,71331,71420,71478,71483,71487,71497,71509,71525,71529,71534
movieID,,,,,,,,,,,,,,,,,,,,,
858,NaN,4.0,NaN,5.0,1.0,2.5,5.0,4.0,NaN,5.0,...,NaN,4.0,5.0,5.0,5.0,5.0,5.0,5.0,NaN,5.0
1214,NaN,5.0,NaN,5.0,3.5,4.0,5.0,4.0,4.5,4.5,...,NaN,3.0,NaN,4.0,NaN,4.0,NaN,5.0,NaN,NaN
2571,4.5,5.0,3.5,5.0,5.0,4.0,NaN,4.0,4.5,4.5,...,5.0,5.0,NaN,4.0,5.0,5.0,3.5,5.0,2.5,4.5


In [140]:
df_user_ratedmovies_timestamps['rating'].value_counts(dropna=False)

4.0    215773
3.0    155918
3.5    150582
4.5     88652
5.0     71680
2.5     62454
2.0     57188
1.0     21535
1.5     18328
0.5     13488
Name: rating, dtype: int64

In [141]:
matrix_zero.fillna(0, inplace=True)

In [142]:
matrix_zero

userID,75,78,170,175,190,267,325,383,476,477,...,71331,71420,71478,71483,71487,71497,71509,71525,71529,71534
movieID,,,,,,,,,,,,,,,,,,,,,
858,0.0,4.0,0.0,5.0,1.0,2.5,5.0,4.0,0.0,5.0,...,0.0,4.0,5.0,5.0,5.0,5.0,5.0,5.0,0.0,5.0
1214,0.0,5.0,0.0,5.0,3.5,4.0,5.0,4.0,4.5,4.5,...,0.0,3.0,0.0,4.0,0.0,4.0,0.0,5.0,0.0,0.0
2571,4.5,5.0,3.5,5.0,5.0,4.0,0.0,4.0,4.5,4.5,...,5.0,5.0,0.0,4.0,5.0,5.0,3.5,5.0,2.5,4.5


In [143]:
# converting the matrix_zero which is a data frame in to a Matrix

Matt =matrix_zero.values

In [144]:
Matt

array([[0. , 4. , 0. , ..., 5. , 0. , 5. ],
       [0. , 5. , 0. , ..., 5. , 0. , 0. ],
       [4.5, 5. , 3.5, ..., 5. , 2.5, 4.5]])

In [145]:
# calculate the sparsity of the matrix

#first calculate the number of non zeros
#  np.count_nonzero(matrix_zero!=0)

#calculate the number of zeros
num_zeros = Matt.size - np.count_nonzero(Matt!=0)
sparsity = num_zeros / Matt.size

In [146]:
sparsity

0.2934504207830223

In [147]:
# It seems that it is NOT very sparse 

In [148]:
# convert matrix to a sparse matrix
sparse_matrix = csr_matrix(Matt)
sparse_matrix

<3x1822 sparse matrix of type '<class 'numpy.float64'>'
	with 3862 stored elements in Compressed Sparse Row format>

In [149]:
print(sparse_matrix)

  (0, 1)	4.0
  (0, 3)	5.0
  (0, 4)	1.0
  (0, 5)	2.5
  (0, 6)	5.0
  (0, 7)	4.0
  (0, 9)	5.0
  (0, 10)	4.0
  (0, 11)	5.0
  (0, 12)	5.0
  (0, 14)	5.0
  (0, 17)	4.0
  (0, 22)	5.0
  (0, 23)	4.5
  (0, 26)	4.0
  (0, 28)	4.0
  (0, 29)	4.5
  (0, 30)	3.5
  (0, 32)	4.5
  (0, 33)	5.0
  (0, 34)	4.5
  (0, 35)	3.0
  (0, 36)	4.5
  (0, 37)	4.0
  (0, 38)	5.0
  :	:
  (2, 1795)	5.0
  (2, 1796)	5.0
  (2, 1797)	5.0
  (2, 1798)	4.0
  (2, 1799)	4.0
  (2, 1800)	4.5
  (2, 1801)	4.0
  (2, 1802)	4.5
  (2, 1803)	4.5
  (2, 1804)	3.5
  (2, 1805)	5.0
  (2, 1806)	0.5
  (2, 1807)	4.0
  (2, 1808)	4.0
  (2, 1809)	2.5
  (2, 1810)	4.5
  (2, 1812)	5.0
  (2, 1813)	5.0
  (2, 1815)	4.0
  (2, 1816)	5.0
  (2, 1817)	5.0
  (2, 1818)	3.5
  (2, 1819)	5.0
  (2, 1820)	2.5
  (2, 1821)	4.5


In [150]:
# Compute a similarity score based on the item information encoded
# cosine_similarity using sklearn library

sk.metrics.pairwise.cosine_similarity(sparse_matrix, Y=None, dense_output=True)

array([[1.        , 0.6626891 , 0.71107806],
       [0.6626891 , 1.        , 0.72501861],
       [0.71107806, 0.72501861, 1.        ]])

In [151]:
# cosine_similarity without using sklearn library

# convert matrix to a sparse matrix
#sparse_matrix = csr_matrix(matrix)

# calculate dot product between rows of the sparse matrix
dot_products = sparse_matrix.dot(sparse_matrix.T)

# calculate Euclidean norm of rows of the sparse matrix
row_norms = np.sqrt(np.sum(np.square(Matt), axis=1))

# calculate cosine similarity
cosine_similarity = dot_products.multiply(1 / row_norms).T.multiply(1 / row_norms)

In [152]:
print(cosine_similarity)

  (0, 0)	1.0
  (0, 1)	0.6626890966346144
  (0, 2)	0.7110780648641414
  (1, 0)	0.6626890966346144
  (1, 1)	0.9999999999999998
  (1, 2)	0.7250186140452017
  (2, 0)	0.7110780648641414
  (2, 1)	0.7250186140452018
  (2, 2)	1.0000000000000002


In [153]:
# convert ratings matrix to binary matrix (We shoul work with matrix that has the null values (I mean matrix not Matt or sparse_matrix) because we need to convert it into a binary matrix 
#in which 1 indicates that a user has rated a movie and 0 indicates that a user has not rated the movie. 

binary_matrix = matrix.notnull().astype('int')

# calculate Jaccard similarity matrix
jaccard_sim_matrix = 1 - pairwise_distances(binary_matrix.values, metric='jaccard')

C:\Users\enabi\anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:1875: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [154]:
jaccard_sim_matrix

array([[1.        , 0.50136986, 0.56473214],
       [0.50136986, 1.        , 0.57142857],
       [0.56473214, 0.57142857, 1.        ]])

In [155]:
# Convert the ratings matrix to a sparse matrix
#sparse_ratings = csr_matrix(matrix.values)

# Compute the correlation matrix using pairwise_distances
correlation_matrix = pairwise_distances(sparse_matrix.toarray(), metric='correlation')

In [156]:
correlation_matrix

array([[0.        , 0.7989134 , 1.07631398],
       [0.7989134 , 0.        , 0.8828587 ],
       [1.07631398, 0.8828587 , 0.        ]])

In [1]:
2 - 1.07631398

0.9236860200000001

In [2]:
!pip install implicit

     ------------------------------------- 634.5/634.5 kB 13.3 MB/s eta 0:00:00
